In [ ]:
import praw
import threading
from json import dumps
from time import sleep
from kafka import KafkaProducer
from concurrent.futures import ThreadPoolExecutor
from prawcore.exceptions import ResponseException


: 

In [ ]:
topic = 'reddit'
clientid = '52.19.145.82:9092'

In [ ]:
producer = KafkaProducer(bootstrap_servers=[clientid], 
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [ ]:
producer.send(topic, value={'test':'value for test'})

In [ ]:
# Define the Reddit API credentials and user agent
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    username='KOBeerose',
    password=password,
    user_agent='Scraper 1.0 by /u/KOBeerose')

In [ ]:
try:
    print("Authenticated as {}".format(reddit.user.me()))
except ResponseException as e:
    print(e)


In [ ]:
for submission in reddit.subreddit("test").hot(limit=10):
    print(submission.title)

In [ ]:
# Define a function to filter submissions by keywords and send them to Kafka
def reddit_stream(keywords):
    for submission in reddit.subreddit('all').search(keywords, limit=None):
        try:
            post = {
                # Output: the submission's id
                'id': submission.id,
                # Output: the submission's title
                'title': submission.title,
                # Output: the submission's text
                'text': submission.selftext,
                # Output: the submission's creation date
                'created_at': submission.created_utc,
                # Output: the submission's score
                'score':submission.score,
                # Output: the submission's author id
                'user_id': submission.author.id,
                # Output: the submission's subreddit
                'subreddit': submission.subreddit.display_name,
                    'user_name': submission.author.name,
                    'followers_count': submission.subreddit.subscribers,
            }
            producer.send(topic, value=post)
            print(f"Sent '{submission.title}' post to Kafka topic '{topic}'")
        except AttributeError as e:
            print(e)
            pass

In [ ]:
keywords = ['bitcoin', 'crypto', 'cryptocurrency', 'blockchain']

In [ ]:
reddit_stream(keywords)

In [ ]:
producer.flush() #clear data from kafka server